In [11]:
import math
def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall=0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN

In [12]:
# !/use/bin/env python
# encoding:utf-8
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.model_selection import train_test_split
import math
import easy_excel
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import *
import sklearn.ensemble
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
import math
from sklearn.model_selection import GridSearchCV
import subprocess
from sklearn.utils import shuffle
import itertools
import joblib
from sklearn.feature_selection import *


In [18]:
import os 
import numpy as np
import pandas as pd
from sklearn.metrics import *
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import *
import re
def generating_integrated_training(file_dir,samples_size):   
    Y=[1 for e in xrange(samples_size)]
    Y2=[0 for e in xrange(samples_size)]
    Y.extend(Y2)
    X=[]
    for root, dirs, files in os.walk(file_dir):  
#         print(root) #当前目录路径  
#         print(dirs) #当前路径下所有子目录  
        for e in files:
            if e=="RFH_KNN_36_dimension_predict.csv" or e=="independent2828_PseEIIP_predict.csv" or e=="independent2828_PCP_predict.csv" or e=="KNN_predict.csv":
                print e
                data=pd.read_csv(root+'/'+e,header=None,index_col=False)
                data=data.values
                positive_=[]
                positive_=[e for index,e in enumerate(data) if data[index,0] == 1 ]
                print len(positive_)
                negative_=[e for index,e in enumerate(data) if data[index,0] == 0 ]
                positive_.extend(negative_)
                data=np.array(positive_)
                data=data[:,1]#设置为probability:2或者predict_label:1
                X.append(data)
    print len(X)
    X=np.array(X).T
    return X,Y
X,Y=generating_integrated_training('/home02/chenhuangrong/december_data',1307)
predict_list=[]
temp_ACC=0
remenber_C=0
remenber_gamma=0
mark_score=[]
for number in xrange(4,5):
#     pd.DataFrame(X).to_csv('/home02/chenhuangrong/integrated_training_files2.csv',header=None,index=False)
    print number
    svc = SVC()
    parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,28)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,28))}
#     parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-5,15,30)), 'gamma':map(lambda x:2**x,np.linspace(-15,5,30))}
    clf = GridSearchCV(svc, parameters, cv=10, n_jobs=12, scoring='accuracy')
    clf.fit(X, Y)
    C=clf.best_params_['C']
    # joblib.dump(clf,'/home02/chenhuangrong/%d_gap_%d_gram.csv'%(gap,gram))
    gamma=clf.best_params_['gamma']
    y_predict=cross_val_predict(SVC(kernel='rbf',C=C,gamma=gamma,),X,Y,cv=10,n_jobs=12)
    y_predict_probability=cross_val_predict(SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,n_jobs=12)
    predict_list.append(y_predict_probability)
    temp_predict_list=np.array(predict_list).T
    X=np.concatenate([X,temp_predict_list],axis=1)
    # print type(y_predict)
    # y_predict_prob=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,n_jobs=-1,method='predict_proba')
    # predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
    # predict_save=np.array(predict_save).T
    # pd.DataFrame(predict_save).to_csv('/home02/chenhuangrong/blending_files/frequency_predict_%d_gram.csv'%(gram),header=None,index=False)
    ROC_AUC_area=roc_auc_score(Y,y_predict)
    ACC=accuracy_score(Y,y_predict)
    mark_score.append([number,ACC])
    print ACC
    if ACC>temp_ACC:
        temp_ACC=ACC
        remenber_C=C
        remenber_gamma=gamma
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
    F1_Score=f1_score(Y, y_predict)
    F_measure=F1_Score
    MCC=matthews_corrcoef(Y, y_predict)
    pos=TP+FN
    neg=FP+TN
    savedata=[[['svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
    print savedata
print temp_ACC
print remenber_C
print remenber_gamma



independent2828_PCP_predict.csv
1307
KNN_predict.csv
1307
independent2828_PseEIIP_predict.csv
1307
RFH_KNN_36_dimension_predict.csv
1307
4
4
0.7467482785
[[['svmC:0.25gamma:0.22560201395', 0.7467482785003825, 0.7475057559478128, 0.7452180566182096, 0.7452180566182096, 0.7482785003825555, 0.7467467106484476, 0.74636015325670502, 0.74636015325670502, 0.49349886813935767, 0.7467482785003825, 974.0, 333.0, 329.0, 978.0, 1307.0, 1307.0]]]
0.7467482785
0.25
0.22560201395


In [56]:
import os 
import numpy as np
import pandas as pd
def generating_integrated_training(file_dir,samples_size):   
    Y=[1 for e in xrange(samples_size)]
    Y2=[0 for e in xrange(samples_size)]
    Y.extend(Y2)
    X=[]
    for root, dirs, files in os.walk(file_dir):  
#         print(root) #当前目录路径  
#         print(dirs) #当前路径下所有子目录  
        for e in files:
            r=re.findall("[a-zA-Z_]+predict.csv",e.strip())
            if r!=[]:
                print e
                data=pd.read_csv(root+'/'+e,header=None,index_col=False)
                data=data.values
                positive_=[e for index,e in enumerate(data) if data[index,0] == 1 ]
                negative_=[e for index,e in enumerate(data) if data[index,0] == 0 ]
                positive_.extend(negative_)
                data=np.array(positive_)
                data=data[:,2]#设置为probability:2或者predict_label:1
                X.append(data)
    print len(X)
    X=np.array(X).T
    return X,Y
X_,Y=generating_integrated_training('/home02/chenhuangrong/december_data',1307)
for select_num in xrange(1,X_.shape[1]):
    F,pval=f_classif(X_,Y)
    idx=np.argsort(F)[::-1]
    select_list=idx[xrange(select_num)]
    X=pd.DataFrame(X_).iloc[:,select_list]
    X=X.values

#     pd.DataFrame(X).to_csv('/home02/chenhuangrong/integrated_training_files2.csv',header=None,index=False)
    print select_num
    print select_list
    svc = svm.SVC()
    parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,7)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,7))}
#     parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-5,15,30)), 'gamma':map(lambda x:2**x,np.linspace(-15,5,30))}
    clf = GridSearchCV(svc, parameters, cv=10, n_jobs=12, scoring='accuracy')
    clf.fit(X, Y)
    C=clf.best_params_['C']
    # joblib.dump(clf,'/home02/chenhuangrong/%d_gap_%d_gram.csv'%(gap,gram))
    print clf.best_score_
    gamma=clf.best_params_['gamma']
    y_predict=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,),X,Y,cv=10,n_jobs=12)
    # print type(y_predict)
    # y_predict_prob=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,n_jobs=-1,method='predict_proba')
    # predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
    # predict_save=np.array(predict_save).T
    # pd.DataFrame(predict_save).to_csv('/home02/chenhuangrong/blending_files/frequency_predict_%d_gram.csv'%(gram),header=None,index=False)
    ROC_AUC_area="%0.4f"%cross_val_score(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=-1).mean()
    ACC=metrics.accuracy_score(Y,y_predict)
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
    F1_Score=metrics.f1_score(Y, y_predict)
    F_measure=F1_Score
    MCC=metrics.matthews_corrcoef(Y, y_predict)
    pos=TP+FN
    neg=FP+TN
    savedata=[[['svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
    print savedata


independent77_PCP_predict.csv
independent97_PseEIIP_predict.csv
independent_PseEIIP_predict.csv
independent22_PseEIIP_predict.csv
independent3_PseEIIP_predict.csv
independent5_PseEIIP_predict.csv
independent_RFHC_GAC_predict.csv
independent2_PCP_predict.csv
independent3030_PseEIIP_predict.csv
independent1414_PseEIIP_predict.csv
position_specific_predict.csv
independent_PCP_predict.csv
KNN_predict.csv
independent11_PseEIIP_predict.csv
independent2828_PseEIIP_predict.csv
independent7_PseEIIP_predict.csv
independent1414_RFHC_GAC_predict.csv
independent1414_PCP_predict.csv
independent9_RFHC_GAC_predict.csv
independent2_PseEIIP_predict.csv
independent2020_PseEIIP_predict.csv
independent77_RFHC_GAC_predict.csv
independent8_RFHC_GAC_predict.csv
23
1
[16]
0.737949502678
[[['svmC:6.34960420787gamma:0.353553390593', 0.73794950267788828, 0.7737676056338029, 0.6725325172149962, 0.6725325172149962, 0.8033664881407804, 0.7350442752076166, 0.71960704052394597, 0.71960704052394597, 0.4800251610233161,

In [62]:
import os 
import numpy as np
import pandas as pd
def generating_integrated_training(file_dir,samples_size):   
    Y=[1 for e in xrange(samples_size)]
    Y2=[0 for e in xrange(samples_size)]
    Y.extend(Y2)
    X=[]
    for root, dirs, files in os.walk(file_dir):  
#         print(root) #当前目录路径  
#         print(dirs) #当前路径下所有子目录  
        i=0
        for e in files:
                
#             r=re.findall("[a-zA-Z_]+predict.csv",e.strip())
#             if r!=[]:
                print e,i
                i=i+1
                data=pd.read_csv(root+'/'+e,header=None,index_col=False)
                data=data.values
                positive_=[e for index,e in enumerate(data) if data[index,0] == 1 ]
                negative_=[e for index,e in enumerate(data) if data[index,0] == 0 ]
                positive_.extend(negative_)
                data=np.array(positive_)
                data=data[:,1]#设置为probability:2或者predict_label:1
                X.append(data)
    print len(X)
    X=np.array(X).T
    return X,Y
X_,Y=generating_integrated_training('/home02/chenhuangrong/december_data/integrated_training',1307)
for select_num in xrange(1,X_.shape[1]):
    F,pval=f_classif(X_,Y)
    idx=np.argsort(F)[::-1]
    select_list=idx[xrange(select_num)]
    X=pd.DataFrame(X_).iloc[:,select_list]
    X=X.values

#     pd.DataFrame(X).to_csv('/home02/chenhuangrong/integrated_training_files2.csv',header=None,index=False)
    print select_num
    print select_list
    svc = svm.SVC()
    parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,7)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,7))}
#     parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-5,15,30)), 'gamma':map(lambda x:2**x,np.linspace(-15,5,30))}
    clf = GridSearchCV(svc, parameters, cv=10, n_jobs=12, scoring='accuracy')
    clf.fit(X, Y)
    C=clf.best_params_['C']
    # joblib.dump(clf,'/home02/chenhuangrong/%d_gap_%d_gram.csv'%(gap,gram))
    print clf.best_score_
    gamma=clf.best_params_['gamma']
    y_predict=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,),X,Y,cv=10,n_jobs=12)
    # print type(y_predict)
    # y_predict_prob=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,n_jobs=-1,method='predict_proba')
    # predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
    # predict_save=np.array(predict_save).T
    # pd.DataFrame(predict_save).to_csv('/home02/chenhuangrong/blending_files/frequency_predict_%d_gram.csv'%(gram),header=None,index=False)
    ROC_AUC_area="%0.4f"%cross_val_score(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=-1).mean()
    ACC=metrics.accuracy_score(Y,y_predict)
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
    F1_Score=metrics.f1_score(Y, y_predict)
    F_measure=F1_Score
    MCC=metrics.matthews_corrcoef(Y, y_predict)
    pos=TP+FN
    neg=FP+TN
    savedata=[[['svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
    print savedata


RFH_predict.csv 0
PseDNC_predict.csv 1
only_3_gram_predict.csv 2
AthMethPre_independent_test__.csv 3
independent2828_PCP_predict.csv 4
TD_predict.csv 5
D_UG_predict.csv 6
PSFNP.csv 7
T_G_U_predict.csv 8
D_CG_predict.csv 9
PSTNP_2gap.csv 10
T_C_G_predict.csv 11
frequency_predict_2_gram.csv 12
CTD_predict.csv 13
PSDNP_2gap.csv 14
only_2_gram_predict.csv 15
D_AC_predict.csv 16
T_C_U_predict.csv 17
only_5_gram_predict.csv 18
frequency_predict_4_gram.csv 19
KNN_predict.csv 20
PSTNP.csv 21
only_4_gram_predict.csv 22
RFH_revised_.csv 23
C_predict.csv 24
T_A_G_predict.csv 25
frequency_predict_3_gram.csv 26
D_predict.csv 27
PSNP.csv 28
D_AU_predict.csv 29
7_DC.csv 30
D_UC_predict.csv 31
3_DC.csv 32
D_AG_predict.csv 33
CT_predict.csv 34
independent2828_PseEIIP_predict.csv 35
PSDNP_1gap.csv 36
only_6_gram_predict.csv 37
PSDNP.csv 38
PSTNP_DS.csv 39
CD_predict.csv 40
independent1414_RFHC_GAC_predict.csv 41
PSNP_PSDNP_1gap_predict.csv 42
frequency_predict_6_gram.csv 43
T_predict.csv 44
6_DC.csv 45


In [42]:
import re
r=re.findall("[a-zA-Z]+predict.csv","fadsiuuhfiuhasdhfahsprdedict.csv")
if r==[]:
    print "d"

d
